In [13]:
import vonage
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import keras

In [14]:
with open('data_20240723.txt', 'r') as f:
    data = f.read()

data_pattern = re.compile(r'(\d{4})\s+(\d{3})\s+(\d{1,2})\s+(\d{1,2})\s+(\d{3})\s+([-]?\d+)\s+(\d+)\s+(\d{3}\.\d)')
matches = data_pattern.findall(data)

In [15]:
columns = ['YEAR', 'DOY', 'HR', 'Kp_index', 'R', 'Dst_index', 'ap_index', 'f10.7_index']
df = pd.DataFrame(matches, columns=columns)

In [16]:
df = df.astype({
    'YEAR': int,
    'DOY': int,
    'HR': int,
    'Kp_index': int,
    'R': int,
    'Dst_index': int,
    'ap_index': int,
    'f10.7_index': float
})

In [17]:
new_df = df[['YEAR', 'R', 'f10.7_index']] 
new_df['TEC'] = np.random.uniform(7, 40, size=len(new_df))
new_df

C:\Users\aadhy\AppData\Local\Temp\ipykernel_19512\165325927.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df['TEC'] = np.random.uniform(7, 40, size=len(new_df))


,YEAR,R,f10.7_index,TEC
0,2024,999,184.2,17.898377
1,2024,999,184.2,25.830671
2,2024,999,184.2,22.885436
3,2024,999,184.2,15.768157
4,2024,999,184.2,12.257191
5,2024,999,184.2,13.642774
6,2024,999,184.2,14.752135
7,2024,999,184.2,17.331907
8,2024,999,184.2,14.098214
9,2024,999,184.2,34.789022


In [18]:
def df_to_X_y2(df, window_size=6):
    df_as_np = df.to_numpy()
    X = []
    y = []
    for i in range(len(df_as_np) - window_size):
        row = [r for r in df_as_np[i:i + window_size]]
        X.append(row)
        label = df_as_np[i + window_size][0]  # Adjust the label indexing as necessary
        y.append(label)
    return np.array(X), np.array(y)

NEW_WINDOW_SIZE = 6
X_new, y_new = df_to_X_y2(new_df, NEW_WINDOW_SIZE)


In [19]:
scaler = MinMaxScaler()

# Fit and transform the new data
X_new_normalized = scaler.fit_transform(X_new.reshape(-1, X_new.shape[-1])).reshape(X_new.shape)
y_new_normalized = scaler.fit_transform(y_new.reshape(-1, y_new.shape[-1])).reshape(y_new.shape)

print(X_new_normalized.shape, y_new_normalized.shape)

(18, 6, 4) (18,)


In [20]:
model = keras.saving.load_model('models/model4/m4.keras')

In [21]:
predictions = model.predict(X_new_normalized)

# Inverse transform the predictions if necessary
#predictions = scaler.inverse_transform(predictions)
predictions = predictions * 550

print(predictions)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 158ms/step
[[33.388885]
 [34.43395 ]
 [42.57383 ]
 [46.117912]
 [51.351295]
 [43.620724]
 [47.78205 ]
 [43.92193 ]
 [40.779873]
 [28.00307 ]
 [33.94581 ]
 [25.457806]
 [31.204422]
 [31.168926]
 [37.740532]
 [33.140358]
 [35.471657]
 [34.468277]]


In [22]:
def send_sms_notification(to_phone, message):
    client = vonage.Client(key='b16ca364', secret='oApPQzh3EZRQBthA')
    sms = vonage.Sms(client)

    responseData = sms.send_message({
        'from': 'VonageAPI',
        'to': to_phone,
        'text': message,
    })

    if responseData["messages"][0]["status"] == "0":
        print("Message sent successfully.")
    else:
        print(f"Message failed with error: {responseData['messages'][0]['error-text']}")

In [23]:
TEC_THRESHOLD = 40.0 

In [24]:
for predicted_tec in predictions:
    if predicted_tec > TEC_THRESHOLD:
        message = f'The predicted TEC value is {predicted_tec}. Communications might be affected.'
        user_phones = ['+919008295292']  
        for phone in user_phones:
            send_sms_notification(phone, message)
        break

Message sent successfully.
